# Lyric baseline

In [1]:
from tqdm import tqdm 

from src.baseline import make_lyric_baseline_fast
from results import victory_odes

for responsion_id in tqdm(victory_odes):
    if responsion_id[0:2] == "ol":
        make_lyric_baseline_fast(f"data/compiled/triads/ht_olympians_triads.xml", responsion_id)
    elif responsion_id[0:2] == "py":
        make_lyric_baseline_fast(f"data/compiled/triads/ht_pythians_triads.xml", responsion_id)
    elif responsion_id[0:2] == "ne":
        make_lyric_baseline_fast(f"data/compiled/triads/ht_nemeans_triads.xml", responsion_id)
    elif responsion_id[0:2] == "is":
        make_lyric_baseline_fast(f"data/compiled/triads/ht_isthmians_triads.xml", responsion_id)


100%|██████████| 100/100 [00:00<00:00, 259.53it/s]



Writing lyric baseline for responsion is01 to data/compiled/baselines/triads/lyric


100%|██████████| 100/100 [00:00<00:00, 379.34it/s]

  5%|▌         | 2/40 [00:01<00:20,  1.89it/s]

Writing lyric baseline for responsion is02 to data/compiled/baselines/triads/lyric


100%|██████████| 100/100 [00:00<00:00, 167.66it/s]



...

Writing lyric baseline for responsion py10 to data/compiled/baselines/triads/lyric


100%|██████████| 100/100 [00:00<00:00, 296.58it/s]



Writing lyric baseline for responsion py11 to data/compiled/baselines/triads/lyric


100%|██████████| 100/100 [00:00<00:00, 412.97it/s]

100%|██████████| 40/40 [00:32<00:00,  1.22it/s]

Writing lyric baseline for responsion py12 to data/compiled/baselines/triads/lyric
